<h1>Hospital Simulation</h1>

In [0]:
"""
### Simulation Model: Multi-Department Hospital Workflow

#### General Description:
This program simulates patient journeys through a hospital composed of seven primary sections: 
Pre-admission, Emergency, Laboratory, Surgery, General Ward, ICU, and CCU. Using discrete-event simulation, 
we aim to evaluate the performance of hospital operations, identify bottlenecks, and analyze patient service patterns. 
The simulation provides insights into system utilization, queue behaviors, and operational efficiency.

---

#### Input Parameters:
1. **Arrival Process**:
   - Patients arrive stochastically based on a Poisson process, with an average inter-arrival time of 20 minutes.
   - Emergency group arrivals are modeled as discrete uniform distributions with group sizes ranging from 2 to 5 people.

2. **Service Time Distributions**:
   - **Pre-surgery**: Uniform distribution (10-25 minutes).
   - **Laboratory**: Exponential, varying based on urgency level.
   - **Surgery**:
     - Simple: Normally distributed (mean = 30.22 mins, variance = 4.9588 mins²).
     - Moderate: Normally distributed (mean = 74.54 mins, variance = 9.9532 mins²).
     - Complex: Normally distributed (mean = 242.031 mins, variance = 63.2745 mins²).
   - **ICU/CCU Stay**: Exponential distribution with a mean of 25 hours.

3. **System Configuration**:
   - Emergency room: 10 beds, limited queue size.
   - Other queues: Unlimited capacity unless otherwise specified.
   - Pre-surgery capacity: 25 beds.
   - ICU and CCU beds: 10 and 5 respectively, subject to temporary power constraints.

---

#### Simulation Objectives:
- Calculate average patient waiting times and lengths of stay.
- Measure queue lengths and probabilities of capacity limits being exceeded.
- Evaluate operational metrics like bed utilization and surgery delays.
- Assess reoperation rates for patients undergoing complex procedures.

---

#### Outputs:
The program generates detailed reports on:
1. **Patient Flow**:
   - System-wide metrics such as throughput, waiting times, and admission delays.
2. **Queue Dynamics**:
   - Average and maximum queue lengths for all departments.
   - Detailed metrics for emergency scenarios.
3. **Resource Utilization**:
   - Real-time utilization rates of beds, laboratories, and operating rooms.

---

#### Notes:
- Power outages simulate real-world challenges, reducing ICU/CCU bed availability to 80%.
- Priority policies ensure that emergency patients are served with minimal delay.
- Analytical reports support hospital managers in optimizing resource allocation.

 
"""


<h2>Important Libraries</h2>

In [5]:
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.stats import t

<h2>Patient Generator Function</h2>

In [ ]:
def generate_random_group(future_event_list ,data, state, clock):
    
    # urgent group arrival
    
    group_number = discrete_uniform(2, 5)
    
    if state['NBEM'] <= 10 - group_number:
        
        state['NBEM'] += group_number
        data['Cumulative Stats']['Service Starters']['Emergency'] += group_number
        
        for i in range(group_number):
            member = attribute_generator(data)
            member['Patient_Type'] = 1
            member['Group_Number'] = group_number 
            member['Arrival_Type'] = 1
            data['Patients_info'][member['Patient_Number']] = member

            e = 1e-7
            data['Patients']['Emergency_arrival'][member['Patient_Number']] = clock + i * e
            data['Patients']['Emergency service begins'][member['Patient_Number']] = clock + i * e
            fel_maker(future_event_list, event_type='lab_arrival', clock=clock + i * e, patient=member)
            
        #generate_random_group(future_event_list, data, state, clock)                
    
    else:
        #generate_random_group(future_event_list, data, state, clock)       
        state['ND'] += group_number


<h2>Attribute Generator Function</h2>

In [ ]:
def attribute_generator(data):
    attributes = {}

    # patient number
    attributes['Patient_Number'] = len(data['Patients_info']) + 1

    # patient type

    attributes['Patient_Type'] = 0

    # surgery type
    surgery_type = ['S', 'M', 'C']
    surgery_probabilities = [0.5, 0.45, 0.05]
    attributes['Surgery_Type'] = random.choices(surgery_type, surgery_probabilities)[0]

    # Complex surgery type
    if attributes['Surgery_Type'] == 'C':
        complex_type = ['CC', 'CU']
        complex_type_probabilities = [0.25, 0.75]
        attributes['Complex_Type'] = random.choices(complex_type, complex_type_probabilities)[0]
    else:
        attributes['Complex_Type'] = 0

    # part
    if attributes['Surgery_Type'] == 'M':
        part = ['ICU', 'CCU', 'G']
        part_probabilities = [0.1, 0.2, 0.7]
        attributes['Part'] = random.choices(part, part_probabilities)[0]
    elif attributes['Surgery_Type'] == 'S':
        attributes['Part'] = 'G'
    else:
        attributes['Part'] = 'ICU' if attributes['Complex_Type'] == 'CU' else 'CCU'


    attributes['Arrival_Type'] = 0
    attributes['Group_Number'] = 0

    # last part default
    attributes['Last_Part'] = ''

    return attributes


In [ ]:
def define_part(part):
    if part == 'G':
        event_type = 'General_arrival'
    elif part == 'ICU':
        event_type = 'ICU_arrival'
    else:
        event_type = 'CCU_arrival'
    return event_type


<h2>Starting State Function</h2>

In [ ]:
def starting_state():
    # State variables
    state = dict()
    state['NQA'] = 0                       # 1-Number of Patients in reservation Queue
    state['NQL'] = 0                       # 2-Number of Non_Urgent Patients in Lab Queue
    state['NUQL'] = 0                      # 3-Number of Urgent Patients in Lab Queue
    state['NUQOR'] = 0                     # 4-Number of Urgent Patients in OR Queue
    state['NQOR'] = 0                      # 5-Number of Non_Urgent Patients in OR Queue
    state['NQCCU'] = 0                     # 6-Number of Patients in CCU Queue
    state['NQICU'] = 0                     # 7-Number of Patients in ICU Queue
    state['NQG'] = 0                       # 8-Number of Patients in General Queue
    state['ND'] = 0                        # 9-Number of leaving Patients in the emergency
    state['NDE'] = 0                       # 10-Number of Dead Patients
    state['NL'] = 0                        # 11-Number of Non_Urgent Patients being served in the lab
    state['NUL'] = 0                       # 12-Number of Urgent Patients being served in the lab
    state['NOR'] = 0                       # 13-Number of Non_Urgent Patients being served in the OR
    state['NUOR'] = 0                      # 14-Number of Urgent Patients being served in the OR
    state['NBPreS'] = 0                    # 15-Number of occupied beds in PreSurgery
    state['NBEM'] = 0                      # 16-Number of occupied beds in emergency room
    state['NBAM'] = 0                      # 17-Number of occupied beds in Ambulance
    state['NBL'] = 0                       # 18-Number of occupied beds in lab
    state['NBOR'] = 0                      # 19-Number of occupied beds in OR
    state['NBCCU'] = 0                     # 20-Number of occupied beds in CCU
    state['NBICU'] = 0                     # 21-Number of occupied beds in ICU
    state['NBG'] = 0                       # 22-Number of occupied beds in General
    state['NES'] = 0                       # 23-Number of served patients
    state['Available_ICU'] = 10            # 24-Number of available beds in CCU due to electricity
    state['Available_CCU'] = 5             # 25-Number of available beds in CCU due to electricity
    state['Reoperation_times'] = 0         # 26-Number of patients who needs reoperation
    state['total_complex_surgery'] = 0     # 27-Number of patients who have had complex surgery

    # saving data
    data = dict()

    data['Last Time Queue Length Changed'] = dict()
    data['Last Time Queue Length Changed']['Emergency Queue'] = 0
    data['Last Time Queue Length Changed']['Reservation Queue'] = 0
    data['Last Time Queue Length Changed']['Non_Urgent Patients in Lab Queue'] = 0
    data['Last Time Queue Length Changed']['Urgent Patients in Lab Queue'] = 0
    data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'] = 0
    data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'] = 0
    data['Last Time Queue Length Changed']['Patients in CCU Queue'] = 0
    data['Last Time Queue Length Changed']['Patients in ICU Queue'] = 0
    data['Last Time Queue Length Changed']['Patients in General Queue'] = 0

    # store all users' data in each queue
    data['Queue Patients'] = dict()
    data['Queue Patients']['Emergency Queue'] = dict()
    data['Queue Patients']['Reservation Queue'] = dict()
    data['Queue Patients']['Non_Urgent Patients in Lab Queue'] = dict()
    data['Queue Patients']['Urgent Patients in Lab Queue'] = dict()
    data['Queue Patients']['Urgent Patients in OR Queue'] = dict()
    data['Queue Patients']['Non_Urgent Patients in OR Queue'] = dict()
    data['Queue Patients']['Patients in CCU Queue'] = dict()
    data['Queue Patients']['Patients in ICU Queue'] = dict()
    data['Queue Patients']['Patients in General Queue'] = dict()

    # last length of each queue
    data['Last Queue Length'] = dict()
    data['Last Queue Length']['Emergency Queue'] = 0
    data['Last Queue Length']['Reservation Queue'] = 0
    data['Last Queue Length']['Non_Urgent Patients in Lab Queue'] = 0
    data['Last Queue Length']['Urgent Patients in Lab Queue'] = 0
    data['Last Queue Length']['Urgent Patients in OR Queue'] = 0
    data['Last Queue Length']['Non_Urgent Patients in OR Queue'] = 0
    data['Last Queue Length']['Patients in CCU Queue'] = 0
    data['Last Queue Length']['Patients in ICU Queue'] = 0
    data['Last Queue Length']['Patients in General Queue'] = 0

    # maximum Queue length
    data['Maximum Queue length'] = dict()
    data['Maximum Queue length']['Emergency Queue'] = 0
    data['Maximum Queue length']['Reservation Queue'] = 0
    data['Maximum Queue length']['Non_Urgent Patients in Lab Queue'] = 0
    data['Maximum Queue length']['Urgent Patients in Lab Queue'] = 0
    data['Maximum Queue length']['Urgent Patients in OR Queue'] = 0
    data['Maximum Queue length']['Non_Urgent Patients in OR Queue'] = 0
    data['Maximum Queue length']['Patients in CCU Queue'] = 0
    data['Maximum Queue length']['Patients in ICU Queue'] = 0
    data['Maximum Queue length']['Patients in General Queue'] = 0

    # Cumulative Stats
    data['Cumulative Stats'] = dict()

    # area under each queue length curve.
    data['Cumulative Stats']['Area Under Queue Length Curve'] = dict()
    data['Cumulative Stats']['Area Under Queue Length Curve']['Emergency Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Reservation Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in Lab Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in Lab Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in OR Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in CCU Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in ICU Queue'] = 0
    data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in General Queue'] = 0

    # area under waiting time for users in each queue
    data['Cumulative Stats']['Area Under Waiting time'] = dict()

    data['Cumulative Stats']['Area Under Waiting time']['Reservation Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Emergency Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in Lab Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in Lab Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in OR Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in OR Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Patients in CCU Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Patients in ICU Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Patients in General Queue'] = 0
    data['Cumulative Stats']['Area Under Waiting time']['Probability of the emergency queue being full'] = 0

    # server busy time
    data['Cumulative Stats']['Server Busy time'] = dict()
    data['Cumulative Stats']['Server Busy time']['Emergency'] = 0
    data['Cumulative Stats']['Server Busy time']['Pre_surgery'] = 0
    data['Cumulative Stats']['Server Busy time']['Lab'] = 0
    data['Cumulative Stats']['Server Busy time']['Operation_Room'] = 0
    data['Cumulative Stats']['Server Busy time']['ICU'] = 0
    data['Cumulative Stats']['Server Busy time']['CCU'] = 0
    data['Cumulative Stats']['Server Busy time']['General'] = 0

    # service starter
    data['Cumulative Stats']['Service Starters'] = dict()
    data['Cumulative Stats']['Service Starters']['Emergency'] = 0
    data['Cumulative Stats']['Service Starters']['Pre_surgery'] = 0
    data['Cumulative Stats']['Service Starters']['Lab'] = 0
    data['Cumulative Stats']['Service Starters']['Operation_Room'] = 0
    data['Cumulative Stats']['Service Starters']['ICU'] = 0
    data['Cumulative Stats']['Service Starters']['CCU'] = 0
    data['Cumulative Stats']['Service Starters']['General'] = 0

    data['Patients'] = dict()  # To track each customer, saving their arrival time, time service begins

    # arrival in events
    data['Patients']['Emergency_arrival'] = dict()
    data['Patients']['presurgery_arrival'] = dict()
    data['Patients']['Lab_arrival'] = dict()
    data['Patients']['Lab_departure'] = dict()
    data['Patients']['Operation_Room_arrival'] = dict()
    data['Patients']['CCU_arrival'] = dict()
    data['Patients']['ICU_arrival'] = dict()
    data['Patients']['General_arrival'] = dict()
    data['Patients']['General_departure'] = dict()

    # service time begins
    data['Patients']['Emergency service begins'] = dict()
    data['Patients']['Pre_surgery service begins'] = dict()
    data['Patients']['Lab service begins'] = dict()
    data['Patients']['Operation Room service begins'] = dict()
    data['Patients']['CCU service begins'] = dict()
    data['Patients']['ICU service begins'] = dict()
    data['Patients']['General service begins'] = dict()

    # save all patients data
    data['Patients_info'] = dict()

    data['Total_System_Duration'] = 0
    
    # Starting FEL
    future_event_list = list()
    first_patient = attribute_generator(data)
    
    future_event_list.append({'Event_Type': 'presurgery_arrival', 'Event_Time': 0,
                              'Patient': first_patient})  # This is an Event Notice
    
    hour = discrete_uniform(1, 30) * 24
    future_event_list.append({'Event_Type': 'power_cutoff', 'Event_Time':hour })
    future_event_list.append({'Event_Type': 'power_return', 'Event_Time':hour + 24 })

    data['Patients_info'][first_patient['Patient_Number']] = first_patient
    
   

    return state, future_event_list, data


<h2> Generating Random Variables Function</h2>

<h3>Uniform Dist.</h3>

In [ ]:
def uniform(a, b):
    r = random.random()
    return a + (b - a) * r

<h3>Exponential Dist.</h3>


In [ ]:
def exponential(lambd):
    r = random.random()
    return -(1 / lambd) * math.log(r)

<h3>Triangular Dist.</h3>

In [ ]:
def triangular(a, b, c):
    r = random.random()
    if r <= 0.5:
        return math.sqrt(2 * r)
    else:
        return 2 - math.sqrt(2 * (1 - r))

<h3>Normal Dist.</h3>

In [ ]:
def normal(mean, std):
    r1 = random.random()
    r2 = random.random()
    z = (math.sqrt(-2 * math.log(r1))) * math.cos(2 * math.pi * r2)
    return mean + std * z

<h3>Discrete Uniform Dist.</h3>

In [ ]:
def discrete_uniform(a, b):
    r = random.random()
    return int(a + ((b + 1) - a) * r)

<h2>Future Event List Maker Function</h2>

In [ ]:
def fel_maker(future_event_list, event_type, clock, patient=None, seed=None):
    # Gets an Event Type
    # Generates activity time for that particular Event Type
    # Creates an event (or more precisely an event notice)
    # Appends the event to FEL
    
    if seed is not None:
        random.seed(seed)    
    
    event_time = 0

    if event_type == 'emergency_arrival':
        event_time = clock + exponential(4)
    elif event_type == 'presurgery_arrival':
        event_time = clock + exponential(1)
    elif event_type == 'lab_arrival':
        if patient['Patient_Type'] == 1:
            event_time = clock + 10/60
        else:
            event_time = clock + 1
    elif event_type == 'lab_departure':
        event_time = clock + uniform(28/60, 32/60)

    elif event_type == 'OperationRoom_arrival':
        if patient['Patient_Type'] == 1:
            event_time = clock + triangular(5/60, 75/60, 100/60)
        else:
            event_time = clock + 2*24

    elif event_type == 'General_arrival':
        if patient['Last_Part'] == 'OR':
            event_time = clock + normal(30.22/60, 4.9588/60) + 10/60
        elif patient['Last_Part'] == 'CCU':
            event_time = clock + exponential(1/25)
        elif patient['Last_Part'] == 'ICU':
            event_time = clock + exponential(1/25)

    elif event_type == 'CCU_arrival':
        if patient['Surgery_Type'] == 'M':
            event_time = clock + normal(74.54/60, 9.9532/60) + 10/60
        elif patient['Surgery_Type'] == 'C':
            event_time = clock + normal(242.031/60, 63.2745/60) + 10/60

    elif event_type == 'ICU_arrival':
        if patient['Surgery_Type'] == 'M':
            event_time = clock + normal(74.54/60, 9.9532/60) + 10/60
        elif patient['Surgery_Type'] == 'C':
            event_time = clock + normal(242.031/60, 63.2745/60) + 10/60

    elif event_type == 'General_departure':
        event_time = clock + exponential(1/50)

    elif event_type == 'power_cutoff':
        event_time = clock + 24

    elif event_type == 'power_return':
        event_time = clock + 30 * 24

    new_event = {'Event_Type': event_type, 'Event_Time': event_time, 'Patient': patient}
    future_event_list.append(new_event)


<h2>Define Events</h2>

<h3>Emergency Arrival Event</h3>

In [ ]:
def emergency_arrival(future_event_list, state, clock, patient, data):

    # single emergency arrival
    if random.random() <= 0.995 :
        data['Patients']['Emergency_arrival'][patient['Patient_Number']] = clock
        if state['NBEM'] < 10:
            state['NBEM'] += 1
            fel_maker(future_event_list, event_type='lab_arrival', clock=clock, patient=patient)

            data['Patients']['Emergency service begins'][patient['Patient_Number']] = clock  # track "every move" of this patient
            data['Cumulative Stats']['Service Starters']['Emergency'] += 1

        elif state['NBEM'] == 10:
            if state['NBAM'] < 10:
                data['Cumulative Stats']['Area Under Queue Length Curve']['Emergency Queue'] += state['NBAM'] * (clock - data['Last Time Queue Length Changed']['Emergency Queue'])
                data['Last Time Queue Length Changed']['Emergency Queue'] = clock
                state['NBAM'] += 1
                data['Queue Patients']['Emergency Queue'][patient['Patient_Number']] = clock  # add this patient to the queue
                data['Maximum Queue length']['Emergency Queue'] = max(data['Maximum Queue length']['Emergency Queue'],state['NBAM'])

            elif state['NBAM'] == 10:
                # exit the hospital
                state['ND'] += 1
                # new edition 
                data['Patients']['Emergency_arrival'].pop(patient['Patient_Number'])

    else:
        # generating the number of patients in group arrival
        generate_random_group(future_event_list, data, state, clock)
        
        
    if random.random() <= 0.75:
        att = attribute_generator(data)
        fel_maker(future_event_list, event_type='presurgery_arrival', clock=clock, patient=att)
        data['Patients_info'][att['Patient_Number']] = att
    else:
        if random.random() <= 0.995:
            att = attribute_generator(data)
            att['Patient_Type'] = 1
            fel_maker(future_event_list, event_type='emergency_arrival', clock=clock, patient=att)
            data['Patients_info'][att['Patient_Number']] = att
        else:
            generate_random_group(future_event_list, data, state, clock)


<h3>Presurgery Arrival Event</h3>

In [ ]:
def presurgery_arrival(future_event_list, state, clock, patient, data):
    data['Patients']['presurgery_arrival'][patient['Patient_Number']] = clock

    if state['NBPreS'] < 25:
        state['NBPreS'] += 1
        data['Patients']['Pre_surgery service begins'][patient['Patient_Number']] = clock  # track "every move" of this patient
        data['Cumulative Stats']['Service Starters']['Pre_surgery'] += 1
        fel_maker(future_event_list, event_type='lab_arrival', clock=clock, patient=patient)


    elif state['NBPreS'] == 25:
        data['Cumulative Stats']['Area Under Queue Length Curve']['Reservation Queue'] += state['NQA'] * (clock - data['Last Time Queue Length Changed']['Reservation Queue'])
        data['Last Time Queue Length Changed']['Reservation Queue'] = clock
        state['NQA'] += 1
        data['Queue Patients']['Reservation Queue'][patient['Patient_Number']] = clock  # add this patient to the queue
        data['Maximum Queue length']['Reservation Queue'] = max(data['Maximum Queue length']['Reservation Queue'],
                                                                state['NQA'])

    if random.random() <= 0.75:
        att = attribute_generator(data)
        fel_maker(future_event_list, event_type='presurgery_arrival', clock=clock, patient=att)
        data['Patients_info'][att['Patient_Number']] = att
    else:
        if random.random() <= 0.995:
            att = attribute_generator(data)
            att['Patient_Type'] = 1
            fel_maker(future_event_list, event_type='emergency_arrival', clock=clock, patient=att)
            data['Patients_info'][att['Patient_Number']] = att
        else:
            generate_random_group(future_event_list, data, state, clock)


<h3>Lab Arrival Event </h3>

In [ ]:
def lab_arrival(future_event_list, state, clock, patient, data):
    data['Patients']['Lab_arrival'][patient['Patient_Number']] = clock
   

    if patient['Patient_Type'] == 1:  # urgent
        if state['NBL'] < 3:
            state['NUL'] += 1
            state['NBL'] += 1
            data['Cumulative Stats']['Service Starters']['Lab'] += 1
            data['Patients']['Lab service begins'][patient['Patient_Number']] = clock
            fel_maker(future_event_list, event_type='lab_departure', clock=clock, patient=patient)

        elif state['NBL'] == 3:

            data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in Lab Queue'] += state['NUQL'] * (clock -data['Last Time Queue Length Changed']['Urgent Patients in Lab Queue'])
            data['Last Time Queue Length Changed']['Urgent Patients in Lab Queue'] = clock
            state['NUQL'] += 1
            data['Queue Patients']['Urgent Patients in Lab Queue'][patient['Patient_Number']] = clock
            data['Maximum Queue length']['Urgent Patients in Lab Queue'] = max(data['Maximum Queue length']['Urgent Patients in Lab Queue'], state['NUQL'])

    else:

        if state['NBL'] < 3:
            state['NL'] += 1
            state['NBL'] += 1
            data['Cumulative Stats']['Service Starters']['Lab'] += 1
            data['Patients']['Lab service begins'][patient['Patient_Number']] = clock
            fel_maker(future_event_list, event_type='lab_departure', clock=clock, patient=patient)

        elif state['NBL'] == 3:
            data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in Lab Queue'] += state['NQL'] * (clock - data['Last Time Queue Length Changed']['Non_Urgent Patients in Lab Queue'])
            data['Last Time Queue Length Changed']['Non_Urgent Patients in Lab Queue'] = clock
            state['NQL'] += 1
            data['Queue Patients']['Non_Urgent Patients in Lab Queue'][patient['Patient_Number']] = clock
            data['Maximum Queue length']['Non_Urgent Patients in Lab Queue'] = max(data['Maximum Queue length']['Non_Urgent Patients in Lab Queue'], state['NQL'])


<h3>Lab Departure Event </h3>

In [ ]:
def lab_departure(future_event_list, state, clock, patient, data):
    data['Patients']['Lab_departure'][patient['Patient_Number']] = clock
    

    if patient['Patient_Type'] == 1:  # urgent
        data['Cumulative Stats']['Server Busy time']['Lab'] += clock - data['Patients']['Lab service begins'][patient['Patient_Number']]
        fel_maker(future_event_list, event_type='OperationRoom_arrival', clock=clock, patient=patient)

        if state['NUQL'] > 0 and len(data['Queue Patients']['Urgent Patients in Lab Queue']) > 0 :
           
            data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in Lab Queue'] += state['NUQL'] * (clock - data['Last Time Queue Length Changed']['Urgent Patients in Lab Queue'])
            data['Last Time Queue Length Changed']['Urgent Patients in Lab Queue'] = clock
            
            state['NUQL'] -= 1
            first_patient_being_served = min(data['Queue Patients']['Urgent Patients in Lab Queue'],
                                             key=data['Queue Patients']['Urgent Patients in Lab Queue'].get)
            data['Patients']['Lab service begins'][first_patient_being_served] = clock
            data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in Lab Queue'] += clock - data['Patients']['Lab_arrival'][first_patient_being_served]
            fel_maker(future_event_list, event_type='lab_departure', clock=clock, patient=data['Patients_info'][first_patient_being_served])
            data['Queue Patients']['Urgent Patients in Lab Queue'].pop(first_patient_being_served, None)

        elif state['NUQL'] == 0:
            if state['NQL'] > 0 and len(data['Queue Patients']['Non_Urgent Patients in Lab Queue']) > 0:
                
                data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in Lab Queue'] += state['NQL'] * (clock - data['Last Time Queue Length Changed']['Non_Urgent Patients in Lab Queue'])
                data['Last Time Queue Length Changed']['Non_Urgent Patients in Lab Queue'] = clock

                state['NQL'] -= 1
                state['NUL'] -= 1
                state['NL'] += 1
                first_patient_being_served = min(data['Queue Patients']['Non_Urgent Patients in Lab Queue'], key=data['Queue Patients']['Non_Urgent Patients in Lab Queue'].get)
                data['Patients']['Lab service begins'][first_patient_being_served] = clock
                data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in Lab Queue'] += clock - data['Patients']['Lab_arrival'][first_patient_being_served]
                fel_maker(future_event_list, event_type='lab_departure', clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Non_Urgent Patients in Lab Queue'].pop(first_patient_being_served, None)

            elif state['NQL'] == 0:
                state['NBL'] -= 1
                state['NUL'] -= 1

    else:

        data['Cumulative Stats']['Server Busy time']['Lab'] += clock - data['Patients']['Lab service begins'][patient['Patient_Number']]
        fel_maker(future_event_list, event_type='OperationRoom_arrival', clock=clock, patient=patient)

        if state['NUQL'] > 0 and len(data['Queue Patients']['Urgent Patients in Lab Queue']) > 0:
            
            data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in Lab Queue'] += state['NUQL'] * (clock - data['Last Time Queue Length Changed']['Urgent Patients in Lab Queue'])
            data['Last Time Queue Length Changed']['Urgent Patients in Lab Queue'] = clock

            state['NUQL'] -= 1
            state['NL'] -= 1
            state['NUL'] += 1

            first_patient_being_served = min(data['Queue Patients']['Urgent Patients in Lab Queue'], key=data['Queue Patients']['Urgent Patients in Lab Queue'].get)
            data['Patients']['Lab service begins'][first_patient_being_served] = clock
            data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in Lab Queue'] += clock - data['Patients']['Lab_arrival'][first_patient_being_served]
            fel_maker(future_event_list, event_type='lab_departure', clock=clock, patient=data['Patients_info'][first_patient_being_served])
            data['Queue Patients']['Urgent Patients in Lab Queue'].pop(first_patient_being_served, None)

        elif state['NUQL'] == 0:
            if state['NQL'] > 0 and len(data['Queue Patients']['Non_Urgent Patients in Lab Queue']) > 0:
                
                
                data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in Lab Queue'] += state['NQL'] * (clock - data['Last Time Queue Length Changed']['Non_Urgent Patients in Lab Queue'])
                data['Last Time Queue Length Changed']['Non_Urgent Patients in Lab Queue'] = clock


                state['NQL'] -= 1
                first_patient_being_served = min(data['Queue Patients']['Non_Urgent Patients in Lab Queue'], key=data['Queue Patients']['Non_Urgent Patients in Lab Queue'].get)
                data['Patients']['Lab service begins'][first_patient_being_served] = clock
                data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in Lab Queue'] += clock - data['Patients']['Lab_arrival'][first_patient_being_served]
                fel_maker(future_event_list, event_type='lab_departure', clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Non_Urgent Patients in Lab Queue'].pop(first_patient_being_served, None)

            elif state['NQL'] == 0:
                state['NBL'] -= 1
                state['NL'] -= 1


<h3>Operation Room Arrival Event</h3>

In [ ]:
def OperationRoom_arrival(future_event_list, state, clock, patient, data):
    data['Patients']['Operation_Room_arrival'][patient['Patient_Number']] = clock
    
    if patient['Patient_Type'] == 1 and patient['Patient_Number'] in data['Patients']['Emergency_arrival']:  # urgent
        data['Cumulative Stats']['Server Busy time']['Emergency'] += clock - data['Patients']['Emergency service begins'][patient['Patient_Number']]
    else:  # Non_urgent
        data['Cumulative Stats']['Server Busy time']['Pre_surgery'] += clock - data['Patients']['Pre_surgery service begins'][patient['Patient_Number']]
    
    if state['NBOR'] == 50:
        if patient['Patient_Type'] == 1:  # urgent
            data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'] += state['NUQOR'] * (clock -data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'])
            data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'] = clock
            state['NUQOR'] += 1
            data['Queue Patients']['Urgent Patients in OR Queue'][patient['Patient_Number']] = clock
            data['Maximum Queue length']['Urgent Patients in OR Queue'] = max(data['Maximum Queue length']['Urgent Patients in OR Queue'], state['NUQOR'])
        else:
            data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in OR Queue'] += state['NQOR'] * (clock-data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'])
            data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'] = clock
            state['NQOR'] += 1
            data['Queue Patients']['Non_Urgent Patients in OR Queue'][patient['Patient_Number']] = clock
            data['Maximum Queue length']['Non_Urgent Patients in OR Queue'] = max(
                data['Maximum Queue length']['Non_Urgent Patients in OR Queue'], state['NQOR'])

    elif state['NBOR'] < 50:
        # attribute last part
        patient['Last_Part'] = 'OR'
        data['Cumulative Stats']['Service Starters']['Operation_Room'] += 1
        data['Patients']['Operation Room service begins'][patient['Patient_Number']] = clock  # track "every move" of this patient

        state['NBOR'] += 1

        if patient['Patient_Type'] == 1:  # urgent
            state['NUOR'] += 1
        else:  # Non_urgent
            state['NOR'] += 1
        # check the Surgery_Type
        if patient['Surgery_Type'] == 'S':
            fel_maker(future_event_list, event_type='General_arrival', clock=clock, patient=patient)
        elif patient['Surgery_Type'] == 'M':
            if patient['Part'] == 'G':
                fel_maker(future_event_list, event_type='General_arrival', clock=clock, patient=patient)
            elif patient['Part'] == 'CCU':
                fel_maker(future_event_list, event_type='CCU_arrival', clock=clock, patient=patient)
            elif patient['Part'] == 'ICU':
                fel_maker(future_event_list, event_type='ICU_arrival', clock=clock, patient=patient)
        else:
            state['total_complex_surgery'] += 1
            if random.random() > 0.1:
                if patient['Complex_Type'] == 'CU':
                    fel_maker(future_event_list, event_type='ICU_arrival', clock=clock, patient=patient)
                else:
                    fel_maker(future_event_list, event_type='CCU_arrival', clock=clock, patient=patient)
            else:
                state['NDE'] += 1
                state['NBOR'] -= 1

                if patient['Patient_Type'] == 1:  # urgent
                    state['NUOR'] -= 1
                else:  # Non_urgent
                    state['NOR'] -= 1

        # second part
        if patient['Patient_Type'] == 1:  # urgent
            # len must be deleted
            if state['NBAM'] > 0 and len(data['Queue Patients']['Emergency Queue']) > 0:
                
                data['Cumulative Stats']['Area Under Queue Length Curve']['Emergency Queue'] += state['NBAM'] * (clock - data['Last Time Queue Length Changed']['Emergency Queue'])
                data['Last Time Queue Length Changed']['Emergency Queue'] = clock
                state['NBAM'] -= 1
                first_patient_being_served = min(data['Queue Patients']['Emergency Queue'], key=data['Queue Patients']['Emergency Queue'].get)
                if state['NBAM'] == 10:
                    data['Cumulative Stats']['Area Under Waiting time'][
                        'Probability of the emergency queue being full'] += clock - data['Patients']['Emergency_arrival'][
                        first_patient_being_served]
                data['Patients']['Emergency service begins'][first_patient_being_served] = clock
                data['Cumulative Stats']['Area Under Waiting time']['Emergency Queue'] += clock - data['Patients'][
                    'Emergency_arrival'][first_patient_being_served]
                fel_maker(future_event_list, event_type='lab_arrival', clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Emergency Queue'].pop(first_patient_being_served, None)
            elif state['NBAM'] == 0:
                state['NBEM'] -= 1
    
        else:  # Non_urgent
            # len must be deleted
            if state['NQA'] > 0 and len(data['Queue Patients']['Reservation Queue']) > 0:
                
                data['Cumulative Stats']['Area Under Queue Length Curve']['Reservation Queue'] += state['NQA'] * (clock - data['Last Time Queue Length Changed']['Reservation Queue'])
                data['Last Time Queue Length Changed']['Reservation Queue'] = clock
                
                state['NQA'] -= 1
                first_patient_being_served = min(data['Queue Patients']['Reservation Queue'],key=data['Queue Patients']['Reservation Queue'].get)
                data['Patients']['Pre_surgery service begins'][first_patient_being_served] = clock
                data['Cumulative Stats']['Area Under Waiting time']['Reservation Queue'] += clock - data['Patients'][
                    'presurgery_arrival'][first_patient_being_served]
                fel_maker(future_event_list, event_type='lab_arrival', clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Reservation Queue'].pop(first_patient_being_served, None)
    
            elif state['NQA'] == 0:
                state['NBPreS'] -= 1
                

<h3>CCU Arrival Event</h3>

In [ ]:
def CCU_arrival(future_event_list, state, clock, patient, data):
    data['Cumulative Stats']['Server Busy time']['Operation_Room'] += clock - data['Patients']['Operation Room service begins'][patient['Patient_Number']]
    # reoperation
    if random.random() <=  0.01:
        state['Reoperation_times'] += 1
        patient['Patient_Type'] = 1
        
        data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'] += state['NUQOR'] * (clock -data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'])
        data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'] = clock

        state['NUQOR'] += 1
        data['Queue Patients']['Urgent Patients in OR Queue'][patient['Patient_Number']] = clock
        data['Patients']['Operation_Room_arrival'][patient['Patient_Number']] = clock

    # dont need reoperation
    else:
        data['Patients']['CCU_arrival'][patient['Patient_Number']] = clock

        if state['NBCCU'] >= state['Available_CCU']:
            data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in CCU Queue'] += state['NQCCU'] * (clock - data['Last Time Queue Length Changed']['Patients in CCU Queue'])
            data['Last Time Queue Length Changed']['Patients in CCU Queue'] = clock
            state['NQCCU'] += 1
            data['Queue Patients']['Patients in CCU Queue'][patient['Patient_Number']] = clock
            data['Maximum Queue length']['Patients in CCU Queue'] = max(data['Maximum Queue length']['Patients in CCU Queue'], state['NQCCU'])

        elif state['NBCCU'] < state['Available_CCU']:
            patient['Last_Part'] = 'CCU'
            state['NBCCU'] += 1
            data['Cumulative Stats']['Service Starters']['CCU'] += 1
            data['Patients']['CCU service begins'][patient['Patient_Number']] = clock
            fel_maker(future_event_list, event_type='General_arrival', clock=clock, patient=patient)

    if patient['Patient_Type'] == 1:  # urgent
        state['NUOR'] -= 1
    else:  # Non_urgent
        state['NOR'] -= 1

    # Second Part
    if state['NUQOR'] > 0 and len(data['Queue Patients']['Urgent Patients in OR Queue']) > 0:
        
        data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'] += state['NUQOR'] * (clock -data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'])
        data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'] = clock
        
        state['NUQOR'] -= 1
        state['NUOR'] += 1

        first_patient_being_served = min(data['Queue Patients']['Urgent Patients in OR Queue'],key=data['Queue Patients']['Urgent Patients in OR Queue'].get)
        data['Patients']['Operation Room service begins'][first_patient_being_served] = clock
        data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in OR Queue'] += clock - data['Patients']['Operation_Room_arrival'][first_patient_being_served]

        # NEW EDITION 
        if  data['Patients_info'][first_patient_being_served]['Surgery_Type'] == 'C':
            if random.random() > 0.1:
                fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Urgent Patients in OR Queue'].pop(first_patient_being_served, None)

            else:
                state['NDE'] += 1
        else:
            fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
            data['Queue Patients']['Urgent Patients in OR Queue'].pop(first_patient_being_served, None)
    
    
    elif state['NUQOR'] == 0:
        if state['NQOR'] > 0 and len(data['Queue Patients']['Non_Urgent Patients in OR Queue']) > 0:
            
            data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in OR Queue'] += state['NQOR'] * (clock-data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'])
            data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'] = clock

            state['NQOR'] -= 1
            state['NOR'] += 1

            first_patient_being_served = min(data['Queue Patients']['Non_Urgent Patients in OR Queue'],
                                             key=data['Queue Patients']['Non_Urgent Patients in OR Queue'].get)
            data['Patients']['Operation Room service begins'][first_patient_being_served] = clock
            data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in OR Queue'] += clock -  data['Patients']['Operation_Room_arrival'][first_patient_being_served]
            
            # NEW EDITION 
            if  data['Patients_info'][first_patient_being_served]['Surgery_Type'] == 'C':
                if random.random() > 0.1:
                    fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                    data['Queue Patients']['Non_Urgent Patients in OR Queue'].pop(first_patient_being_served, None)

                else:
                    state['NDE'] += 1
            else:
                
                fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Non_Urgent Patients in OR Queue'].pop(first_patient_being_served, None)
        
        
        elif state['NQOR'] == 0:
            state['NBOR'] -= 1


<h3>ICU Arrival Event</h3>

In [ ]:
def ICU_arrival(future_event_list, state, clock, patient, data):
    data['Cumulative Stats']['Server Busy time']['Operation_Room'] += clock - data['Patients']['Operation Room service begins'][patient['Patient_Number']]

    if random.random() <= 0.01:
        state['Reoperation_times'] += 1
        patient['Patient_Type'] = 1
        data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'] += state['NUQOR'] * (clock -data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'])
        data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'] = clock

        state['NUQOR'] += 1
        data['Queue Patients']['Urgent Patients in OR Queue'][patient['Patient_Number']] = clock
        data['Patients']['Operation_Room_arrival'][patient['Patient_Number']] = clock


    else:
        data['Patients']['ICU_arrival'][patient['Patient_Number']] = clock
  

        if state['NBICU'] >= state['Available_ICU']:
            data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in ICU Queue'] += state['NQICU'] * (
                    clock - data['Last Time Queue Length Changed']['Patients in ICU Queue'])
            data['Last Time Queue Length Changed']['Patients in ICU Queue'] = clock
            state['NQICU'] += 1
            data['Queue Patients']['Patients in ICU Queue'][patient['Patient_Number']] = clock
            data['Maximum Queue length']['Patients in ICU Queue'] = max(
                data['Maximum Queue length']['Patients in ICU Queue'], state['NQICU'])


        elif state['NBICU'] < state['Available_ICU']:
            patient['Last_Part'] = 'ICU'
            state['NBICU'] += 1
            data['Cumulative Stats']['Service Starters']['ICU'] += 1
            data['Patients']['ICU service begins'][patient['Patient_Number']] = clock
            fel_maker(future_event_list, event_type="General_arrival", clock=clock, patient=patient)

    if patient['Patient_Type'] == 1:  # urgent
        state['NUOR'] -= 1
    else:  # Non_urgent
        state['NOR'] -= 1

    # Second Part
    if state['NUQOR'] > 0 and len(data['Queue Patients']['Urgent Patients in OR Queue']) > 0:
        
        data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'] += state['NUQOR'] * (clock -data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'])
        data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'] = clock
 
        state['NUQOR'] -= 1
        state['NUOR'] += 1

        first_patient_being_served = min(data['Queue Patients']['Urgent Patients in OR Queue'],
                                         key=data['Queue Patients']['Urgent Patients in OR Queue'].get)
        data['Patients']['Operation Room service begins'][first_patient_being_served] = clock
        data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in OR Queue'] += clock - data['Patients'][
            'Operation_Room_arrival'][first_patient_being_served]
        
        # NEW EDITION 
        if  data['Patients_info'][first_patient_being_served]['Surgery_Type'] == 'C':
            if random.random() > 0.1:
                fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Urgent Patients in OR Queue'].pop(first_patient_being_served, None)

            else:
                state['NDE'] += 1
        else:
            fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
            data['Queue Patients']['Urgent Patients in OR Queue'].pop(first_patient_being_served, None)
    

    elif state['NUQOR'] == 0:
        if state['NQOR'] > 0 and len(data['Queue Patients']['Non_Urgent Patients in OR Queue']) > 0:
            
            data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in OR Queue'] += state['NQOR'] * (clock-data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'])
            data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'] = clock
 
            state['NQOR'] -= 1
            state['NOR'] += 1

            first_patient_being_served = min(data['Queue Patients']['Non_Urgent Patients in OR Queue'],
                                             key=data['Queue Patients']['Non_Urgent Patients in OR Queue'].get)
            data['Patients']['Operation Room service begins'][first_patient_being_served] = clock
            data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in OR Queue'] += clock - data['Patients']['Operation_Room_arrival'][first_patient_being_served]
            
            # NEW EDITION 
            if  data['Patients_info'][first_patient_being_served]['Surgery_Type'] == 'C':
                if random.random() > 0.1:
                    fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                    data['Queue Patients']['Non_Urgent Patients in OR Queue'].pop(first_patient_being_served, None)

                else:
                    state['NDE'] += 1
            else:
                
                fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Non_Urgent Patients in OR Queue'].pop(first_patient_being_served, None)
                                                                                          
        elif state['NQOR'] == 0:
            state['NBOR'] -= 1

<h3>General Arrival Event</h3>

In [ ]:
def General_arrival(future_event_list, state, clock, patient, data):
    data['Patients']['General_arrival'][patient['Patient_Number']] = clock

    if patient['Last_Part'] == 'CCU':
        
        data['Cumulative Stats']['Server Busy time']['CCU'] += clock - data['Patients']['CCU service begins'][patient['Patient_Number']]
    elif patient['Last_Part'] == 'ICU':
        data['Cumulative Stats']['Server Busy time']['ICU'] += clock - data['Patients']['ICU service begins'][patient['Patient_Number']]

    elif patient['Last_Part'] == 'OR':
        data['Cumulative Stats']['Server Busy time']['Operation_Room'] += clock - data['Patients']['Operation Room service begins'][patient['Patient_Number']]

    if state['NBG'] == 40:
        data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in General Queue'] += state['NQG'] * (clock - data['Last Time Queue Length Changed']['Patients in General Queue'])
        data['Last Time Queue Length Changed']['Patients in General Queue'] = clock
        state['NQG'] += 1
        data['Queue Patients']['Patients in General Queue'][patient['Patient_Number']] = clock
        data['Maximum Queue length']['Patients in General Queue'] = max(
            data['Maximum Queue length']['Patients in General Queue'], state['NQG'])

        if patient['Last_Part'] == 'CCU':
            state['NBCCU'] -= 1
        elif patient['Last_Part'] == 'ICU':
            state['NBICU'] -= 1
        else:
            state['NBOR'] -= 1
        
    
    elif state['NBG'] < 40:
        data['Cumulative Stats']['Service Starters']['General'] += 1
        data['Patients']['General service begins'][patient['Patient_Number']] = clock
        state['NBG'] += 1

        # Last_Part
        if patient['Last_Part'] == 'CCU':
            fel_maker(future_event_list, event_type='General_departure', clock=clock, patient=patient)
            if state['NQCCU'] == 0:
                state['NBCCU'] -= 1
            elif state['NQCCU'] > 0:
                
                data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in CCU Queue'] += state['NQCCU'] * (clock - data['Last Time Queue Length Changed']['Patients in CCU Queue'])
                data['Last Time Queue Length Changed']['Patients in CCU Queue'] = clock

                state['NQCCU'] -= 1
                first_patient_being_served = min(data['Queue Patients']['Patients in CCU Queue'],
                                                 key=data['Queue Patients']['Patients in CCU Queue'].get)
                data['Patients']['CCU service begins'][first_patient_being_served] = clock
                data['Cumulative Stats']['Area Under Waiting time']['Patients in CCU Queue'] += clock -  data['Patients']['CCU_arrival'][ first_patient_being_served]

                data['Patients_info'][first_patient_being_served]['Last_Part'] = 'CCU'
                
                fel_maker(future_event_list, event_type='General_arrival', clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Queue Patients']['Patients in CCU Queue'].pop(first_patient_being_served, None)

        elif patient['Last_Part'] == 'ICU':
            fel_maker(future_event_list, event_type='General_departure', clock=clock, patient=patient)
            
            if state['NQICU'] == 0:
                state['NBICU'] -= 1
            elif state['NQICU'] > 0:
                
                data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in ICU Queue'] += state['NQICU'] * (clock - data['Last Time Queue Length Changed']['Patients in ICU Queue'])
                data['Last Time Queue Length Changed']['Patients in ICU Queue'] = clock

                state['NQICU'] -= 1
                first_patient_being_served = min(data['Queue Patients']['Patients in ICU Queue'],
                                                 key=data['Queue Patients']['Patients in ICU Queue'].get)
                data['Patients']['ICU service begins'][first_patient_being_served] = clock
               
                data['Cumulative Stats']['Area Under Waiting time']['Patients in ICU Queue'] += clock - data['Patients']['ICU_arrival'][first_patient_being_served]
                
                fel_maker(future_event_list, event_type='General_arrival', clock=clock, patient=data['Patients_info'][first_patient_being_served])
                data['Patients_info'][first_patient_being_served]['Last_Part'] = 'ICU'
                data['Queue Patients']['Patients in ICU Queue'].pop(first_patient_being_served, None)

        elif patient['Last_Part'] == 'OR':

            if patient['Patient_Type'] == 1:  # urgent
                state['NUOR'] -= 1
            else:  # Non_urgent
                state['NOR'] -= 1
            fel_maker(future_event_list, event_type='General_departure', clock=clock, patient=patient)

            if state['NUQOR'] > 0 and len(data['Queue Patients']['Urgent Patients in OR Queue']) > 0:
                data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'] += state['NUQOR'] * (clock -data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'])
                data['Last Time Queue Length Changed']['Urgent Patients in OR Queue'] = clock

                state['NUQOR'] -= 1
                state['NUOR'] += 1

                first_patient_being_served = min(data['Queue Patients']['Urgent Patients in OR Queue'],key=data['Queue Patients']['Urgent Patients in OR Queue'].get)

                data['Patients']['Operation Room service begins'][first_patient_being_served] = clock
                data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in OR Queue'] += clock - data['Patients']['Operation_Room_arrival'][first_patient_being_served]

                # NEW EDITION 
                if  data['Patients_info'][first_patient_being_served]['Surgery_Type'] == 'C':
                    if random.random() > 0.1:
                        fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                        data['Queue Patients']['Urgent Patients in OR Queue'].pop(first_patient_being_served, None)

                    else:
                        state['NDE'] += 1
                else:
                   fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                   data['Queue Patients']['Urgent Patients in OR Queue'].pop(first_patient_being_served, None)
    

            elif state['NUQOR'] == 0:
                if state['NQOR'] > 0 and len(data['Queue Patients']['Non_Urgent Patients in OR Queue']) > 0:
                    data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in OR Queue'] += state['NQOR'] * (clock-data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'])
                    data['Last Time Queue Length Changed']['Non_Urgent Patients in OR Queue'] = clock

                    state['NQOR'] -= 1
                    state['NOR'] += 1
                    first_patient_being_served = min(data['Queue Patients']['Non_Urgent Patients in OR Queue'],key=data['Queue Patients']['Non_Urgent Patients in OR Queue'].get)
                    data['Patients']['Operation Room service begins'][first_patient_being_served] = clock
                    data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in OR Queue'] += clock -data['Patients']['Operation_Room_arrival'][first_patient_being_served]
                    
                    # NEW EDITION 
                    if  data['Patients_info'][first_patient_being_served]['Surgery_Type'] == 'C':
                        if random.random() > 0.1:
                            fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                            data['Queue Patients']['Non_Urgent Patients in OR Queue'].pop(first_patient_being_served, None)

                        else:
                            state['NDE'] += 1
                    else:
                        fel_maker(future_event_list, event_type=define_part(data['Patients_info'][first_patient_being_served]["Part"]), clock=clock, patient=data['Patients_info'][first_patient_being_served])
                        data['Queue Patients']['Non_Urgent Patients in OR Queue'].pop(first_patient_being_served, None)
 
                elif state['NQOR'] == 0:
                    state['NBOR'] -= 1

<h3>General Departure Event</h3>

In [ ]:
def General_departure(future_event_list, state, clock, patient, data):
    
    data['Patients']['General_departure'][patient['Patient_Number']] = clock
    data['Cumulative Stats']['Server Busy time']['General'] += clock - data['Patients']['General service begins'][
        patient['Patient_Number']]
    
    if patient['Patient_Type'] == 1 and patient['Patient_Number'] in data['Patients']['Emergency_arrival']:
        data['Total_System_Duration'] += data['Patients']['General_departure'][patient['Patient_Number']] - data['Patients']['Emergency_arrival'][patient['Patient_Number']]
        
    else:
        data['Total_System_Duration'] += data['Patients']['General_departure'][patient['Patient_Number']] - data['Patients']['presurgery_arrival'][patient['Patient_Number']]
        
    state['NES'] += 1
    if state['NQG'] > 0:
        
        data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in General Queue'] += state['NQG'] * (clock - data['Last Time Queue Length Changed']['Patients in General Queue'])
        data['Last Time Queue Length Changed']['Patients in General Queue'] = clock
     
        state['NQG'] -= 1
        first_patient_being_served = min(data['Queue Patients']['Patients in General Queue'],key=data['Queue Patients']['Patients in General Queue'].get)
        data['Patients']['General service begins'][first_patient_being_served] = clock
        data['Cumulative Stats']['Area Under Waiting time']['Patients in General Queue'] += clock - data['Patients']['General_arrival'][first_patient_being_served]
        fel_maker(future_event_list, event_type='General_departure', clock=clock, patient=data['Patients_info'][first_patient_being_served])       
        data['Queue Patients']['Patients in General Queue'].pop(first_patient_being_served, None)

    elif state['NQG'] == 0:
        state['NBG'] -= 1

<h3>Power CutOff Event</h3>

In [ ]:
def power_cutoff(future_event_list, state, clock, data):

    state['Available_ICU'] = 8
    state['Available_CCU'] = 4


<h3>Power Return Event</h3>

In [ ]:
def power_return(future_event_list, state, clock, data):
    
    state['Available_ICU'] = 10
    state['Available_CCU'] = 5

<h2> Excel Output Function </h2>

In [ ]:
def create_row(step, current_event, state, data, future_event_list):
    # This function will create a list, which will eventually become a row of the output Excel file

    sorted_fel = sorted(future_event_list, key=lambda x: x['Event_Time'])

    if current_event['Event_Type'] !='power_cutoff' and current_event['Event_Type'] != 'power_return' :
        row = [step, current_event['Event_Time'], current_event['Event_Type'], current_event['Patient']]
           
    else:    
        row = [step, current_event['Event_Time'], current_event['Event_Type'], 'Patient']
   
    row.extend(list(state.values()))
    row.extend(list(data['Last Time Queue Length Changed'].values()))
    row.extend(list(data['Queue Patients'].values()))
    row.extend(list(data['Last Queue Length'].values()))
    row.extend(list(data['Maximum Queue length'].values()))
    row.extend(list(data['Cumulative Stats']['Area Under Queue Length Curve'].values()))
    row.extend(list(data['Cumulative Stats']['Area Under Waiting time'].values()))
    row.extend(list(data['Cumulative Stats']['Server Busy time'].values()))
    row.extend(list(data['Cumulative Stats']['Service Starters'].values()))
    row.extend(list(data['Patients'].values()))



    for event in sorted_fel:
        row.append(event['Event_Time'])
        row.append(event['Event_Type'])
        if event['Event_Type'] !='power_cutoff' and event['Event_Type'] != 'power_return' :
            row.append(event['Patient'])
           
        else:
            row.append('No Patient Need')
    return row


def justify(table):
    # This function adds blanks to short rows in order to match their lengths to the maximum row length

    # Find maximum row length in the table
    row_max_len = 0
    for row in table:
        if len(row) > row_max_len:
            row_max_len = len(row)

    # For each row, add enough blanks
    for row in table:
        row.extend([""] * (row_max_len - len(row)))


def create_main_header(state, data):
    # This function creates the main part of header (returns a list)
    # A part of header which is used for future events will be created in create_excel()

    # Header consists of ...
    # 1. Step, Clock, Event Type and Event Customer
    header = ['Step', 'Clock', 'Event Type', 'Event Patient']
    header.extend(list(state.keys()))
    header.extend(list(data['Last Time Queue Length Changed'].keys()))
    header.extend(list(data['Queue Patients'].keys()))
    header.extend(list(data['Last Queue Length'].keys()))
    header.extend(list(data['Maximum Queue length'].keys()))
    header.extend(list(data['Cumulative Stats']['Area Under Queue Length Curve'].keys()))
    header.extend(list(data['Cumulative Stats']['Area Under Waiting time'].keys()))
    header.extend(list(data['Cumulative Stats']['Server Busy time'].keys()))
    header.extend(list(data['Cumulative Stats']['Service Starters'].keys()))
    header.extend(list(data['Patients'].keys()))
    
    return header


def create_excel(table, header):
    # This function creates and fine-tunes the Excel output file

    # Find length of each row in the table
    row_len = len(table[0])

    # Find length of header (header does not include cells for fel at this moment)
    header_len = len(header)

    # row_len exceeds header_len by (max_fel_length * 3) (Event Type, Event Time & Customer for each event in FEL)
    # Extend the header with 'Future Event Time', 'Future Event Type', 'Future Event Customer'
    # for each event in the fel with maximum size
    i = 1
    for col in range((row_len - header_len) // 3):
        header.append('Future Event Time ' + str(i))
        header.append('Future Event Type ' + str(i))
        header.append('Future Event Patient ' + str(i))
        i += 1

    # Dealing with the output
    # First create a pandas DataFrame
    df = pd.DataFrame(table, columns=header, index=None)

    # Create a handle to work on the Excel file
    writer = pd.ExcelWriter('Hospital_Sim_Output.xlsx', engine='xlsxwriter')

    # Write out the Excel file to the hard drive
    df.to_excel(writer, sheet_name='Hospital Simulation Output', header=False, startrow=1, index=False)

    # Use the handle to get the workbook (just library syntax, can be found with a simple search)
    workbook = writer.book

    # Get the sheet you want to work on
    worksheet = writer.sheets['Hospital Simulation Output']

    # Create a cell-formatter object (this will be used for the cells in the header, hence: header_formatter!)
    header_formatter = workbook.add_format()

    # Define whatever format you want
    header_formatter.set_align('center')
    header_formatter.set_align('vcenter')
    header_formatter.set_font('Times New Roman')
    header_formatter.set_bold('True')

    # Write out the column names and apply the format to the cells in the header row
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_formatter)

    for i, width in enumerate(get_col_widths(df)):
        worksheet.set_column(i - 1, i - 1, width)

    # Create a cell-formatter object for the body of excel file
    main_formatter = workbook.add_format()
    main_formatter.set_align('center')
    main_formatter.set_align('vcenter')
    main_formatter.set_font('Times New Roman')

    # Apply the format to the body cells
    for row in range(1, len(df) + 1):
        worksheet.set_row(row, None, main_formatter)

    # Save your edits
    writer.close()


def get_col_widths(dataframe):

    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]


<h2>Metrics Calculation</h2>

In [ ]:
metric_dict = {
    "Average_Lenght_of_System_Duration": [],
    "The_Probability_of_Emergency_Being_Full": [],
    "Maximum_Queue_length_Emergency_Queue": [],
    "Maximum_Queue_length_Reservation_Queue": [],
    "Maximum_Queue_length_Non_Urgent_Patients_in_Lab_Queue": [],
    "Maximum_Queue_length_Urgent_Patients_in_Lab_Queue": [],
    "Maximum_Queue_length_Urgent_Patients_in_OR_Queue": [],
    "Maximum_Queue_length_Non_Urgent_Patients_in_OR_Queue": [],
    "Maximum_Queue_length_in_CCU_Queue": [],
    "Maximum_Queue_length_in_ICU_Queue": [],
    "Maximum_Queue_length_in_General_Queue": [],
    "Average_Queue_length_Emergency_Queue" : [],
    "Average_Queue_length_Reservation_Queue" : [],
    "Average_Queue_length_Non_Urgent_Patients_in_Lab_Queue" : [],
    "Average_Queue_length_Urgent_Patients_in_Lab_Queue" : [],
    "Average_Queue_length_Urgent_Patients_in_OR_Queue" : [],
    "Average_Queue_length_Non_Urgent_Patients_in_OR_Queue" : [],
    "Average_Queue_length_Patients_in_CCU_Queue" : [],
    "Average_Queue_length_Patients_in_ICU_Queue" : [],
    "Average_Queue_length_Patients_in_General_Queue": [],
    "Average_Waiting_Time_in_Emergency_Queue": [],
    "Average_Waiting_Time_in_Reservation_Queue": [],
    "Average_Waiting_Time_in_Lab_Queue": [],
    "Average_Waiting_Time_in_OR_Queue": [],
    "Average_Waiting_Time_in_CCU_Queue": [],
    "Average_Waiting_Time_in_ICU_Queue": [],
    "Average_Waiting_Time_in_General_Queue": [],
    "Average_Number_of_Reoperation": [],
    "Emergency_Bed_Utilization": [],
    "Pre_surgery_Bed_Utilization": [],
    "Lab_Utilization": [],
    "Operation_Room_Bed_Utilization": [],
    "ICU_Bed_Utilization": [],
    "CCU_Bed_Utilization": [],
    "General_Bed_Utilization": [],
    "Average_Number_of_Dead_Patients": []
}

def safe_division(numerator, denominator):
    """
    Performs division safely. Returns 0 if the denominator is 0.
    """
    return numerator / denominator if denominator != 0 else 0

def Metrics_Calculation(state, data, simulation_time):
    Average_Lenght_of_System_Duration = safe_division(data['Total_System_Duration'], len(data['Patients']['General_departure']))
    The_Probability_of_Emergency_Being_Full = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Emergency Queue'], simulation_time)
    
    # Max Queue Length
    Maximum_Queue_length_Emergency_Queue = data['Maximum Queue length']['Emergency Queue'] 
    Maximum_Queue_length_Reservation_Queue = data['Maximum Queue length']['Reservation Queue'] 
    Maximum_Queue_length_Non_Urgent_Patients_in_Lab_Queue = data['Maximum Queue length']['Non_Urgent Patients in Lab Queue'] 
    Maximum_Queue_length_Urgent_Patients_in_Lab_Queue = data['Maximum Queue length']['Urgent Patients in Lab Queue'] 
    Maximum_Queue_length_Urgent_Patients_in_OR_Queue = data['Maximum Queue length']['Urgent Patients in OR Queue'] 
    Maximum_Queue_length_Non_Urgent_Patients_in_OR_Queue = data['Maximum Queue length']['Non_Urgent Patients in OR Queue'] 
    Maximum_Queue_length_in_CCU_Queue = data['Maximum Queue length']['Patients in CCU Queue'] 
    Maximum_Queue_length_in_ICU_Queue = data['Maximum Queue length']['Patients in ICU Queue'] 
    Maximum_Queue_length_in_General_Queue = data['Maximum Queue length']['Patients in General Queue'] 
    
    # Average Queue Length
    Average_Queue_length_Emergency_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Emergency Queue'], simulation_time)
    Average_Queue_length_Reservation_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Reservation Queue'], simulation_time)
    Average_Queue_length_Non_Urgent_Patients_in_Lab_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in Lab Queue'], simulation_time)
    Average_Queue_length_Urgent_Patients_in_Lab_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in Lab Queue'], simulation_time)
    Average_Queue_length_Urgent_Patients_in_OR_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Urgent Patients in OR Queue'], simulation_time)
    Average_Queue_length_Non_Urgent_Patients_in_OR_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Non_Urgent Patients in OR Queue'], simulation_time)
    Average_Queue_length_Patients_in_CCU_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in CCU Queue'], simulation_time)
    Average_Queue_length_Patients_in_ICU_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in ICU Queue'], simulation_time)
    Average_Queue_length_Patients_in_General_Queue = safe_division(data['Cumulative Stats']['Area Under Queue Length Curve']['Patients in General Queue'], simulation_time)
    
    # Average Waiting Time
    Average_Waiting_Time_in_Emergency_Queue = safe_division(data['Cumulative Stats']['Area Under Waiting time']['Emergency Queue'], data['Cumulative Stats']['Service Starters']['Emergency'])
    Average_Waiting_Time_in_Reservation_Queue = safe_division(data['Cumulative Stats']['Area Under Waiting time']['Reservation Queue'], data['Cumulative Stats']['Service Starters']['Pre_surgery'])
    Average_Waiting_Time_in_Lab_Queue = safe_division(data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in Lab Queue'] +  data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in Lab Queue'], data['Cumulative Stats']['Service Starters']['Lab'])
    Average_Waiting_Time_in_OR_Queue = safe_division(data['Cumulative Stats']['Area Under Waiting time']['Urgent Patients in OR Queue'] + data['Cumulative Stats']['Area Under Waiting time']['Non_Urgent Patients in OR Queue'], data['Cumulative Stats']['Service Starters']['Operation_Room'])
    Average_Waiting_Time_in_CCU_Queue = safe_division(data['Cumulative Stats']['Area Under Waiting time']['Patients in CCU Queue'], data['Cumulative Stats']['Service Starters']['CCU'])
    Average_Waiting_Time_in_ICU_Queue = safe_division(data['Cumulative Stats']['Area Under Waiting time']['Patients in ICU Queue'], data['Cumulative Stats']['Service Starters']['ICU'])
    Average_Waiting_Time_in_General_Queue = safe_division(data['Cumulative Stats']['Area Under Waiting time']['Patients in General Queue'], data['Cumulative Stats']['Service Starters']['General'])

    # Average number of reoperation for patients with complex surgery
    Average_Number_of_Reoperation = safe_division(state['Reoperation_times'], state['total_complex_surgery'])
    
    # Bed Utilization
    Emergency_Bed_Utilization = safe_division(data['Cumulative Stats']['Server Busy time']['Emergency'], (simulation_time * 10))
    Pre_surgery_Bed_Utilization = safe_division(data['Cumulative Stats']['Server Busy time']['Pre_surgery'], (simulation_time * 25))
    Lab_Utilization = safe_division(data['Cumulative Stats']['Server Busy time']['Lab'], (simulation_time * 3))
    Operation_Room_Bed_Utilization = safe_division(data['Cumulative Stats']['Server Busy time']['Operation_Room'], (simulation_time * 50))
    ICU_Bed_Utilization = safe_division(data['Cumulative Stats']['Server Busy time']['ICU'], (simulation_time * 10))
    CCU_Bed_Utilization = safe_division(data['Cumulative Stats']['Server Busy time']['CCU'], (simulation_time * 5))
    General_Bed_Utilization = safe_division(data['Cumulative Stats']['Server Busy time']['General'], (simulation_time * 40))
    
    # Average number of Dead patients
    Average_Number_of_Dead_Patients = safe_division(state['NDE'], len(data['Patients']['General_departure']))
    
    # add the metrics to the dictionary (this is for interval calculation)
    
    metric_dict["Average_Lenght_of_System_Duration"].append(Average_Lenght_of_System_Duration)
    metric_dict["The_Probability_of_Emergency_Being_Full"].append(The_Probability_of_Emergency_Being_Full)

    # Maximum Queue Lengths
    metric_dict["Maximum_Queue_length_Emergency_Queue"].append(Maximum_Queue_length_Emergency_Queue)
    metric_dict["Maximum_Queue_length_Reservation_Queue"].append(Maximum_Queue_length_Reservation_Queue)
    metric_dict["Maximum_Queue_length_Non_Urgent_Patients_in_Lab_Queue"].append(Maximum_Queue_length_Non_Urgent_Patients_in_Lab_Queue)
    metric_dict["Maximum_Queue_length_Urgent_Patients_in_Lab_Queue"].append(Maximum_Queue_length_Urgent_Patients_in_Lab_Queue)
    metric_dict["Maximum_Queue_length_Urgent_Patients_in_OR_Queue"].append(Maximum_Queue_length_Urgent_Patients_in_OR_Queue)
    metric_dict["Maximum_Queue_length_Non_Urgent_Patients_in_OR_Queue"].append(Maximum_Queue_length_Non_Urgent_Patients_in_OR_Queue)
    metric_dict["Maximum_Queue_length_in_CCU_Queue"].append(Maximum_Queue_length_in_CCU_Queue)
    metric_dict["Maximum_Queue_length_in_ICU_Queue"].append(Maximum_Queue_length_in_ICU_Queue)
    metric_dict["Maximum_Queue_length_in_General_Queue"].append(Maximum_Queue_length_in_General_Queue)
    
    # Average Queue Lengths
    metric_dict["Average_Queue_length_Emergency_Queue"].append(Average_Queue_length_Emergency_Queue)
    metric_dict["Average_Queue_length_Reservation_Queue"].append(Average_Queue_length_Reservation_Queue)
    metric_dict["Average_Queue_length_Non_Urgent_Patients_in_Lab_Queue"].append(Average_Queue_length_Non_Urgent_Patients_in_Lab_Queue)
    metric_dict["Average_Queue_length_Urgent_Patients_in_Lab_Queue"].append(Average_Queue_length_Urgent_Patients_in_Lab_Queue)
    metric_dict["Average_Queue_length_Urgent_Patients_in_OR_Queue"].append(Average_Queue_length_Urgent_Patients_in_OR_Queue)
    metric_dict["Average_Queue_length_Non_Urgent_Patients_in_OR_Queue"].append(Average_Queue_length_Non_Urgent_Patients_in_OR_Queue)
    metric_dict["Average_Queue_length_Patients_in_CCU_Queue"].append(Average_Queue_length_Patients_in_CCU_Queue)
    metric_dict["Average_Queue_length_Patients_in_ICU_Queue"].append(Average_Queue_length_Patients_in_ICU_Queue)
    metric_dict["Average_Queue_length_Patients_in_General_Queue"].append(Average_Queue_length_Patients_in_General_Queue)
    
    # Average Waiting Times
    metric_dict["Average_Waiting_Time_in_Emergency_Queue"].append(Average_Waiting_Time_in_Emergency_Queue)
    metric_dict["Average_Waiting_Time_in_Reservation_Queue"].append(Average_Waiting_Time_in_Reservation_Queue)
    metric_dict["Average_Waiting_Time_in_Lab_Queue"].append(Average_Waiting_Time_in_Lab_Queue)
    metric_dict["Average_Waiting_Time_in_OR_Queue"].append(Average_Waiting_Time_in_OR_Queue)
    metric_dict["Average_Waiting_Time_in_CCU_Queue"].append(Average_Waiting_Time_in_CCU_Queue)
    metric_dict["Average_Waiting_Time_in_ICU_Queue"].append(Average_Waiting_Time_in_ICU_Queue)
    metric_dict["Average_Waiting_Time_in_General_Queue"].append(Average_Waiting_Time_in_General_Queue)

    # Other Metrics
    metric_dict["Average_Number_of_Reoperation"].append(Average_Number_of_Reoperation)
    metric_dict["Average_Number_of_Dead_Patients"].append(Average_Number_of_Dead_Patients)
    
    # Bed Utilizations
    metric_dict["Emergency_Bed_Utilization"].append(Emergency_Bed_Utilization)
    metric_dict["Pre_surgery_Bed_Utilization"].append(Pre_surgery_Bed_Utilization)
    metric_dict["Lab_Utilization"].append(Lab_Utilization)
    metric_dict["Operation_Room_Bed_Utilization"].append(Operation_Room_Bed_Utilization)
    metric_dict["ICU_Bed_Utilization"].append(ICU_Bed_Utilization)
    metric_dict["CCU_Bed_Utilization"].append(CCU_Bed_Utilization)
    metric_dict["General_Bed_Utilization"].append(General_Bed_Utilization)
    
    
    # Printing the metrics in a structured format
    print("\n=== Metrics Calculation Results ===\n")
    
    # Maximum Queue Length
    print("Maximum Queue Lengths:")
    print(f"  Emergency Queue: {Maximum_Queue_length_Emergency_Queue}")
    print(f"  Reservation Queue: {Maximum_Queue_length_Reservation_Queue}")
    print(f"  Non-Urgent Patients in Lab Queue: {Maximum_Queue_length_Non_Urgent_Patients_in_Lab_Queue}")
    print(f"  Urgent Patients in Lab Queue: {Maximum_Queue_length_Urgent_Patients_in_Lab_Queue}")
    print(f"  Urgent Patients in OR Queue: {Maximum_Queue_length_Urgent_Patients_in_OR_Queue}")
    print(f"  Non-Urgent Patients in OR Queue: {Maximum_Queue_length_Non_Urgent_Patients_in_OR_Queue}")
    print(f"  Patients in CCU Queue: {Maximum_Queue_length_in_CCU_Queue}")
    print(f"  Patients in ICU Queue: {Maximum_Queue_length_in_ICU_Queue}")
    print(f"  Patients in General Queue: {Maximum_Queue_length_in_General_Queue}\n")

    # Average Queue Length
    print("Average Queue Lengths:")
    print(f"  Emergency Queue: {Average_Queue_length_Emergency_Queue}")
    print(f"  Reservation Queue: {Average_Queue_length_Reservation_Queue}")
    print(f"  Non-Urgent Patients in Lab Queue: {Average_Queue_length_Non_Urgent_Patients_in_Lab_Queue}")
    print(f"  Urgent Patients in Lab Queue: {Average_Queue_length_Urgent_Patients_in_Lab_Queue}")
    print(f"  Urgent Patients in OR Queue: {Average_Queue_length_Urgent_Patients_in_OR_Queue}")
    print(f"  Non-Urgent Patients in OR Queue: {Average_Queue_length_Non_Urgent_Patients_in_OR_Queue}")
    print(f"  Patients in CCU Queue: {Average_Queue_length_Patients_in_CCU_Queue}")
    print(f"  Patients in ICU Queue: {Average_Queue_length_Patients_in_ICU_Queue}")
    print(f"  Patients in General Queue: {Average_Queue_length_Patients_in_General_Queue}\n")

    # Average Waiting Time
    print("Average Waiting Times:")
    print(f"  Emergency Queue: {Average_Waiting_Time_in_Emergency_Queue:}")
    print(f"  Reservation Queue: {Average_Waiting_Time_in_Reservation_Queue:}")
    print(f"  Lab Queue: {Average_Waiting_Time_in_Lab_Queue:}")
    print(f"  OR Queue: {Average_Waiting_Time_in_OR_Queue:}")
    print(f"  CCU Queue: {Average_Waiting_Time_in_CCU_Queue:}")
    print(f"  ICU Queue: {Average_Waiting_Time_in_ICU_Queue:}")
    print(f"  General Queue: {Average_Waiting_Time_in_General_Queue:}\n")

    # Other Metrics
    print("Other Metrics:")
    print(f"  Average Length of System Duration: {Average_Lenght_of_System_Duration:.2f}")
    print(f"  Probability of Emergency Being Full: {The_Probability_of_Emergency_Being_Full:.2f}")
    print(f"  Average Number of Reoperation: {Average_Number_of_Reoperation:.2f}")
    print(f"  Average Number of Dead Patients: {Average_Number_of_Dead_Patients:.2f}\n")

    # Bed Utilization
    print("Bed Utilizations:")
    print(f"  Emergency: {Emergency_Bed_Utilization:%}")
    print(f"  Pre-surgery: {Pre_surgery_Bed_Utilization:%}")
    print(f"  Lab: {Lab_Utilization:%}")
    print(f"  Operation Room: {Operation_Room_Bed_Utilization:%}")
    print(f"  ICU: {ICU_Bed_Utilization:%}")
    print(f"  CCU: {CCU_Bed_Utilization:%}")
    print(f"  General: {General_Bed_Utilization:%}")    

<h2>Simulation Function </h2>

In [ ]:
def simulation(simulation_time):
    state, future_event_list, data = starting_state()
    
    clock = 0
    table = []
    step = 1
    future_event_list.append({'Event_Type': 'End of simulation', 'Event_Time': simulation_time, 'Patient': None})
    while clock < simulation_time:
        sorted_fel = sorted(future_event_list, key=lambda x: x['Event_Time'])
        current_event = sorted_fel[0]  # Find imminent event
        clock = current_event['Event_Time']
        if current_event['Event_Type'] != 'power_cutoff' and current_event['Event_Type'] != 'power_return' :
            patient = current_event['Patient']
        
        if clock < simulation_time:  # until we have not met the "End of Simulation" event
            if current_event['Event_Type'] == 'emergency_arrival':
                emergency_arrival(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'presurgery_arrival':
                presurgery_arrival(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'lab_arrival':
                lab_arrival(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'lab_departure':
                lab_departure(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'OperationRoom_arrival':
                OperationRoom_arrival(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'CCU_arrival':
                CCU_arrival(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'ICU_arrival':
                ICU_arrival(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'General_arrival':
                General_arrival(future_event_list, state, clock, patient, data)
            elif current_event["Event_Type"] == "General_departure":
                General_departure(future_event_list, state, clock, patient, data)
            elif current_event['Event_Type'] == 'power_cutoff':
                power_cutoff(future_event_list, state, clock, data)
            elif current_event['Event_Type'] == 'power_return':
                power_return(future_event_list, state, clock, data)

            future_event_list.remove(current_event)

        else:
            future_event_list.clear()
            
        table.append(create_row(step, current_event, state, data, future_event_list))
        step += 1
    print('-------------------------------------------------------------------------------------------------')
    
    #excel_main_header = create_main_header(state, data)
    #justify(table)
    #create_excel(table, excel_main_header)
    
    print('Simulation Ended!')
    Metrics_Calculation(state, data, simulation_time)
    #return data


<h2>Confidence Interval Function</h2>

In [ ]:
def calculate_point_and_interval_estimates(metric_dict, confidence_level=0.95):
    results = {}
    alpha = 1 - confidence_level

    for metric, values in metric_dict.items():
        n_replications = 15  
        if n_replications > 1:  
            mean = np.mean(values)  
            std_dev = np.std(values, ddof=1)  
            t_value = t.ppf(1 - alpha / 2, df=n_replications-1)  
            margin_of_error = t_value * (std_dev / np.sqrt(n_replications))  
            confidence_interval = (mean - margin_of_error, mean + margin_of_error) 

            results[metric] = {
                "Mean": mean,
                "Confidence Interval": confidence_interval
            }
        else:
            results[metric] = {
                "Mean": None,
                "Confidence Interval": None
            }

    return results


results = calculate_point_and_interval_estimates(metric_dict)


for metric, stats in results.items():
    print(f"{metric}:")
    print(f"  Mean: {stats['Mean']}")
    print(f"  Confidence Interval: {stats['Confidence Interval']}")
    print()
